# 🧠 AI-Powered Business Requirement Chatbot
This notebook allows users to:
- Convert natural language or meeting audio into business requirements
- Tag as core/additional features
- View full requirement logs
- Export fully formatted BRDs with industry-standard templates

## 📦 Step 1: Install Required Libraries

In [ ]:
!pip install gradio python-docx openai-whisper

## 🧠 Step 2: Define Requirement Logic and Template BRD Generator

In [ ]:

import re
from docx import Document
from datetime import datetime

history = []

def convert_to_business_requirement(text, label="Core Feature"):
    requirements = []
    for sentence in re.split(r'[.?!]', text):
        if sentence.strip():
            requirement = f"- [{label}] The system shall {sentence.strip().lower()}."
            requirements.append(requirement)
    history.extend(requirements)
    return "\n".join(requirements)

def show_history():
    return "\n".join(history)

def export_to_word():
    doc = Document()
    doc.add_heading("Business Requirements Export", 0)
    doc.add_paragraph(f"Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    for req in history:
        doc.add_paragraph(req, style='List Bullet')
    path = "/mnt/data/Business_Requirements_Export.docx"
    doc.save(path)
    return path

def export_full_brd_template():
    doc = Document()
    doc.add_heading("Business Requirements Document (BRD)", 0)
    doc.add_paragraph(f"Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    sections = {
        "1. Executive Summary": "Provide a high-level overview of the project, including its purpose, objectives, and the business need it addresses.",
        "2. Project Objectives": "Clearly define the specific, measurable goals the project aims to achieve.",
        "3. Project Scope": "Outline the boundaries of the project, detailing what is included and excluded.",
        "4. Business Requirements": "\n".join(history),
        "5. Key Stakeholders": "Identify all individuals or groups with a vested interest in the project, including their roles and responsibilities.",
        "6. Project Constraints": "Document any limitations or restrictions, such as budget, time, technology, or resources.",
        "7. Cost-Benefit Analysis": "Analyze the anticipated costs against the expected benefits to justify the project's viability.",
        "8. Current Process": "Describe the existing process and its shortcomings.",
        "9. Proposed Process": "Detail the new process and how it addresses current issues.",
        "10. Functional Requirements": "Specify the functionalities the system must have.",
        "11. Non-Functional Requirements": "Outline requirements related to performance, security, usability, etc.",
        "12. Financial Statements": "Provide detailed financial projections and budgets.",
        "13. Resources": "List the human, technical, and material resources required.",
        "14. Schedule, Timeline, and Deadlines": "Present a detailed project timeline with milestones.",
        "15. Assumptions": "State any assumptions made during the planning phase.",
        "16. Glossary": "Define terms and acronyms used in the document.",
        "17. References": "Cite all sources and reference materials.",
        "18. Appendix": "Include any additional supporting information."
    }
    for title, content in sections.items():
        doc.add_heading(title, level=2)
        doc.add_paragraph(content)
    path = "/mnt/data/Business_Requirements_Full_Template.docx"
    doc.save(path)
    return path


## 🎙️ Step 3: Transcribe Meeting Audio

In [ ]:

import whisper
model = whisper.load_model("base")

def transcribe_audio(audio_file, label="Core Feature"):
    result = model.transcribe(audio_file.name)
    return convert_to_business_requirement(result["text"], label)


## 🖼️ Step 4: Interactive Gradio UI

In [ ]:

import gradio as gr

text_input = gr.Interface(
    fn=lambda text, label: convert_to_business_requirement(text, label),
    inputs=[
        gr.Textbox(label="Describe a business process or requirement", lines=4),
        gr.Textbox(label="Feature Label (e.g. Core Feature, Additional Feature)", value="Core Feature")
    ],
    outputs=gr.Textbox(label="Generated Requirements"),
    title="🧠 Text to Business Requirement"
)

audio_input = gr.Interface(
    fn=lambda audio, label: transcribe_audio(audio, label),
    inputs=[
        gr.Audio(type="filepath", label="Upload meeting audio"),
        gr.Textbox(label="Feature Label (e.g. Core Feature, Additional Feature)", value="Core Feature")
    ],
    outputs=gr.Textbox(label="Generated Requirements from Audio"),
    title="🎙️ Audio to Requirement Generator"
)

history_display = gr.Interface(
    fn=show_history,
    inputs=[],
    outputs=gr.Textbox(label="📜 Requirement History Log"),
    title="📚 Full Requirement Log"
)

export_docx = gr.Interface(
    fn=export_to_word,
    inputs=[],
    outputs=gr.File(label="Download Requirements Docx"),
    title="📤 Export Raw Requirement Log"
)

export_brd = gr.Interface(
    fn=export_full_brd_template,
    inputs=[],
    outputs=gr.File(label="Download Full BRD Template"),
    title="📄 Export BRD Template"
)

gr.TabbedInterface(
    [text_input, audio_input, history_display, export_docx, export_brd],
    ["Text Input", "Audio Input", "View All Requirements", "Raw Export", "Full BRD Template"]
).launch()
